# PHO Database Project

### Background

* Monthly Master Medical Staff (MMS) list comes out;
* Craig manually adds any new providers to the CIN database;
* It's frustrating & error-prone; we want this to be easier

### What We Want

* Make a list of providers to add to the database
* Make a deduplicated list of healthcare entities and locations from the Master Medical Staff list
* match those against the database


### Assumptions

* All entities come with a TIN; i.e. I haven't considered any rows without a TIN when working with entities.
* All locations come with a street address; i.e. I haven't considered any rows without a street address when working with locations. 


## 0. Basic Stuff

In [1]:
# import libraries
import numpy as np
import pandas as pd

# read files
entities = pd.read_csv('kn_entities.csv')
locations = pd.read_csv('kn_locations.csv')
providers = pd.read_csv('kn_providers.csv')
# For now, we need to specify the sheet names because the .xlsx file contains summary sheets 
sheets = ['CAD','CHX','GRY','KMHC','MAN','MMC','OMH','POMH'] # GO BACK TO MACK LATER; THEY DON'T HAVE NPI'S LOL
MMS = pd.read_excel('master_medical_staff_list_200601.xlsx',sheet_name=sheets)

For now, MMS is a dictionary where the sheet names are the indices and the dataframes are the values.
The last several rows of each sheet is a summary table that is irrelevant to this project but they will be dropped as we clean the dataframe. 

## 1. Make a list of providers to add to the database

We'll make a list of distinct providers using __first name, last name, NPI__.

1. Rename three columns in `providers` (firstname, lastname, npi) so that it matches the ones in `MMS`; extract those three columns
2. For each sheet in `MMS`, 
    1. extract the three columns `Last Name`, `First Name`, `NPI`
    2. Find out who is in the sheet but not in `providers`
    3. Use a sheet name to populate a new column 'Hospital affiliation'
3. export to a new .csv file `new_providers.csv`

In [2]:
# 1. rename three columns in providers & extract those three columns
colnames = {'Provider Name: First': 'First Name',
           'Provider Name: Last': 'Last Name',
           'Provider NPI': 'NPI'}
providers = providers.rename(columns=colnames)
providers = providers[['Last Name','First Name','NPI']]

In [3]:
# (this will be a list of distinct providers who are in MMS but not in providers)
new_providers = pd.DataFrame(columns = ['Last Name','First Name','NPI'])

# 2. iterate through each affiliation 
for affiliation in MMS:
    df = MMS[affiliation]
    # 2A. extract the three relevant columns
    df = df[['Last Name','First Name','NPI']]
    # 2B. compare MMS and providers, and add any distinct provider to new_providers
    diff = df.merge(providers,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
    diff = diff.dropna() # assuming that those without NPI or name are irrelevant...
    # 2C. Use a sheet name to populate a new column 'Hospital affiliation'
    diff['Affiliation'] = affiliation
    new_providers = new_providers.append(diff)

# 4. export to a new .csv file    
new_providers.to_csv('new_providers.csv',index=False)

## 2. Make a deduplicated list of healthcare entities from the Master Medical Staff list; match with the DB

Similar to #1, but for entities...

1. Rename columns in the database to match MMS(`Entity TIN`-->`Tax ID`)
2. Extract relevant parts from `MMS`
    1. Drop all the rows that don't have a `TIN` value.
    2. Choose columns `TIN`, `Practice`, `Practice Address`, `City, State Zip`
    3. Compare the values with the TINs in `entities`
    4. Add to a new dataframe (`new_entities`) whatever entities that are in MMS but not in database
3. Export to a new .csv file (`new_entities.csv`)

In [4]:
# 1. rename columns in entities so that they match MMS
colnames = {'Entity TIN': 'Tax ID'} # add more later
entities = entities.rename(columns=colnames)
entities = entities[['Entity Legal Name','Tax ID']]
# typecast to str so that we can join with MMS later
entities['Tax ID'] = entities['Tax ID'].astype(str).str[0:9]
entities.head()

,Entity Legal Name,Tax ID
0,Active Chiropractic of Cadillac,474680795
1,"Advance Pathology Services, PC",208238099
2,"Advanced Optometry, PLLC",382137907
3,Allergy and Asthma Specialists of Cadillac,383588887
4,"Andrew S. Riemer, DO PC",383156438


In [5]:
new_entities = pd.DataFrame(columns = ['Practice','Tax ID','Practice Address','Zip'])

# 2. extract relevant parts from MMS
for affiliation in MMS:
    df = MMS[affiliation]
    # 2A. select rows that have TIN
    df = df[df['Tax ID'].notna()]
    # 2B. select relevant columns (Can add more later if needed)
    df = df[['Practice','Tax ID','Practice Address','City, State Zip']]
    if df.empty==False: # if there is any entity in MMS to check for,
        df['Tax ID'] = df['Tax ID'].str.replace("-","")
        pattern = r"([0-9]{9})"
        df['TIN'] = df['Tax ID'].str.extract(pattern)
        df = df[df['TIN'].notna()]
        # 2C. left join, leaving only the entities that are in MMS but not in database
        diff = df.merge(entities,indicator = True, how='left',on='Tax ID')
        diff = diff[diff['_merge']=='left_only']
        # 2D. append the deduplicated list of entities to the new dataframe
        new_entities = new_entities.append(diff,ignore_index=True)

new_entities = new_entities[['Practice','Tax ID','Practice Address','City, State Zip']]        

# 3. export the dataframe to a new .csv file    
new_entities.to_csv('new_entities.csv',index=False)

### ISSUES:

* __*`MAN` doesn't have any Tax ID recorded; does this allow me to assume that they are no new entity, or should I come up with  a way to find deduplicated entities from `MAN` too?*__

### Task 3: Make a deduplicated list of locations from the Master Medical Staff list
* *(__locations file is not  cleaned --> ask Adam for a new, cleaned file with no NPI?__)*


#### GAME PLAN

__From `MMS`:__
1. Extract `Practice`, `Practice Address`, `City, State Zip` (for now) (*Drop NANs*)
2. Split the column `City, State Zip` into `City`, `State`, and `Zip`
3. Standaradize `Practice Address`:
    1. Get rid of all the dots and commas
    2. Put it in all caps
    3. Avenue --> Ave; Street-->St; Drive-->Dr; Road--> Rd; Highway-->Hwy, etc.
4. Rename columns: `Practice` --> `Name_MMS`; `Practice Address`--> `Address_MMS`; `Zip`-->`Zip_MMS`; etc.
5. deduplicate

__From `locations`:__
1. Extract `Location Name`, `Physical Address: Street 1`, `Physical Address: Zip` (for now) (*Drop NANs-__if there is any NANs drop the entire row, for now__*)
2. Standardize `Physical Address: Street 1` the same way we did for `MMS`
3. Rename columns: `Location Name` --> `Name_DB`; `Physical Address: Street 1`--> `Address_DB`; `Physical Address: Zip`--> `Zip_DB`; etc.

__With the cleaned dataframes:__
1. Extract rows from `MMS` that are not in `locations` based on `Address_MMS`/`Address_DB` and `Zip_MMS`/`Zip_DB`
2. Put in a new dataframe `new_locations` (*Columns: Name_MMS, Name_DB, Address, City, State, Zip (for now)*)
3. Export the dataframe into a new .csv file `new_locations.csv`


In [6]:
# Working with DB - locations

# 1. Extract `Location Name`, `Physical Address: Street 1`, `Physical Address: Zip` (for now) (*Drop NANs*)
locations = locations[['Location Name','Physical Address: Street 1','Physical Address: Zip']]
locations = locations.dropna(how='any')

# 2. Rename columns: `Location Name` --> `Name_DB`; `Physical Address: Street 1`--> `Address_DB`; `Physical Address: Zip`--> `Zip_DB`; etc.
newcols = {'Location Name':'Name_DB','Physical Address: Street 1':'Address_DB','Physical Address: Zip': 'Zip_DB'}
locations = locations.rename(columns=newcols)

# 3. Standardize `Address_DB` the same way we did for `MMS`
# Get rid of all the dots commas etc etc
locations['Address_DB'] = locations['Address_DB'].str.replace('\W',' ')
# Also get rid of unnecessary whitespaces
locations['Address_DB'] = locations['Address_DB'].str.replace('\s+',' ').str.strip()
# Put it in all caps
locations['Address_DB'] = locations['Address_DB'].str.upper()
# Avenue --> Ave; Street-->St; Drive-->Dr; Road--> Rd; Highway-->Hwy, etc.
locations['Address_DB'] = locations['Address_DB'].str.replace('AVENUE','AVE')
locations['Address_DB'] = locations['Address_DB'].str.replace('STREET','ST')
locations['Address_DB'] = locations['Address_DB'].str.replace('DRIVE','DR')
locations['Address_DB'] = locations['Address_DB'].str.replace('ROAD','RD')
locations['Address_DB'] = locations['Address_DB'].str.replace('HIGHWAY','HWY')
locations['Address_DB'] = locations['Address_DB'].str.replace('TRAIL','TR')
locations['Address_DB'] = locations['Address_DB'].str.replace('SUITE','STE')

# 4. Clean location names
locations['Name_DB'] = locations['Name_DB'].str.replace('\W',' ').str.replace('\s+',' ').str.strip()

# preview
locations.head()

,Name_DB,Address_DB,Zip_DB
0,Active Chiropractic of Cadillac,119 N SHELBY ST,49601
1,Advanced Foot and Ankle Center Cadillac,8805 PINE RIDGE DR,49601
2,Advanced Foot and Ankle Center Manistee 118483...,1860 E PARKDALE AVE STE 2,49660
3,Advanced Foot and Ankle Center Traverse City,1225 FRONT ST STE 200,49684
4,Advanced Optometry,120 PALUSTER ST,49601


In [7]:
# Working with MMS

locations_MMS = pd.DataFrame(columns = ['Name_MMS','Address_MMS','City','State','Zip_MMS'])

for affiliation in MMS:
    df = MMS[affiliation]
    # 1. Extract `Practice`, `Practice Address`, `City, State Zip` (for now)
    df = df[['Practice', 'Practice Address', 'City, State Zip']]
    # 2. Split the column `City, State Zip` into `City`, `State`, and `Zip`
    df[['City','StateZip']] = df['City, State Zip'].str.split(',',expand=True)
    df[['State','Zip']] = df['StateZip'].str.strip().str.split(expand=True)
    df = df[['Practice', 'Practice Address', 'City', 'State', 'Zip']]
        #print(df.head())
    # 3. Standaradize `Practice Address`:
        # 3A. Get rid of all the dots and commas etc 
    df['Practice Address'] = df['Practice Address'].str.replace('\W',' ')
    # Also get rid of unnecessary whitespaces
    df['Practice Address'] = df['Practice Address'].str.replace('\s+',' ').str.strip()
        # 3B. Put it in all caps
    df['Practice Address'] = df['Practice Address'].str.upper()
        # 3C. Avenue --> Ave; Street-->St; Drive-->Dr; Road--> Rd; Highway-->Hwy, etc.
    df['Practice Address'] = df['Practice Address'].str.replace('AVENUE','AVE')
    df['Practice Address'] = df['Practice Address'].str.replace('STREET','ST')
    df['Practice Address'] = df['Practice Address'].str.replace('DRIVE','DR')
    df['Practice Address'] = df['Practice Address'].str.replace('ROAD','RD')
    df['Practice Address'] = df['Practice Address'].str.replace('HIGHWAY','HWY')
    df['Practice Address'] = df['Practice Address'].str.replace('TRAIL','TR')
    df['Practice Address'] = df['Practice Address'].str.replace('SUITE','STE')
    # 4. Rename columns: `Practice` --> `Name_MMS`; `Practice Address`--> `Address_MMS`; `Zip`-->`Zip_MMS`; etc.
    colnames = {'Practice': 'Name_MMS', 'Practice Address': 'Address_MMS', 'Zip': 'Zip_MMS'}
    df = df.rename(columns=colnames)
    locations_MMS = locations_MMS.append(df)

# 5. deduplicate    
locations_MMS = locations_MMS.drop_duplicates()    
    
print(locations_MMS.info())    
locations_MMS.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 381 entries, 0 to 25
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name_MMS     380 non-null    object
 1   Address_MMS  380 non-null    object
 2   City         380 non-null    object
 3   State        380 non-null    object
 4   Zip_MMS      379 non-null    object
dtypes: object(5)
memory usage: 17.9+ KB
None


/Users/hymnkim/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Name_MMS,Address_MMS,City,State,Zip_MMS
0,Munson Healthcare Cadillac Anesthesia,400 HOBART ST,Cadillac,MI,49601
2,"Chowdhury MD, PLLC",8795 PINE RIDGE DR,Cadillac,MI,49601
3,Munson Healthcare Cadillac Cancer & Infusion C...,400 HOBART ST,Cadillac,MI,49601
4,"Family Practice of Cadillac, PC",827 E DIVISION,Cadillac,MI,49601
5,American Healthcare Staffing Association,10126 E CHERRY BEND RD,Traverse City,MI,49684


In [8]:
# __With the cleaned dataframes:__
# 1. Extract rows from `MMS` that are not in `locations` based on `Address_MMS`/`Address_DB` and `Zip_MMS`/`Zip_DB`
new_locations = locations_MMS.merge(locations, indicator=True, how='outer',
                                    left_on='Address_MMS',right_on='Address_DB')
# new_locations = new_locations[new_locations['_merge']=='both']
new_locations = new_locations[new_locations['_merge']=='left_only']
#new_locations = new_locations[(new_locations['_merge']=='left_only') | (new_locations['_merge']=='right_only')]
new_locations = new_locations.drop_duplicates(subset='Name_MMS')
#print(new_locations.info())
#print(new_locations.head(10))

#diff = df.merge(entities,indicator = True, how='left',on='Tax ID')
#        diff = diff[diff['_merge']=='left_only']

# 2. Put in a new dataframe `new_locations` (*Columns: Name_MMS, Name_DB, Address, City, State, Zip (for now)*)
new_locations = new_locations[['Name_MMS','Address_MMS','City','State','Zip_MMS']]

# 3. Export the dataframe into a new .csv file `new_locations.csv`
new_locations.to_csv('new_locations.csv',index=False)

Observations:

* the list is not entirely of new list bc of minor variations in the address (i.e. some have STE# or not have St, Rd, etc)
    For example, Family Practice of Cadillac is already in the database but with a more specific address ('827 E DIVISION STE 2').


Possible solutions:

* When deduplicating `locations_MMS`, make the longest address/longest name absorb the shorter ones (i.e. leave only the entries with the most information)
* When merging the two dataframes `locations_MMS` and `locations`, use the street address but look for 'containment', not 'exact match'. 
    1. i.e. if the entry in `Address_MMS` contains that in `Address_DB`, then either
        1. record both address and include them 
        2. consider that location already in the DB and don't include in the unique lists
    2. i.e. if the entry in `Address_DB` contains that in `Address_MMS`, then 
        1. consider that location already in the DB and don't include in the unique lists

## Ending Note

* Run this thing but check `MACK` separately.
* The sheet affiliated with `MAN` doesn't have any Tax ID recorded for any of the practices; hence no entities were added from this affiliation. If we want to get entities from `MAN` we should come up with some other method. 

## *++++below are just random stuff i've been trying++++*

In [ ]:
# populate a new boolean column 'MMS>DB' that says 'True' if MMS address contains DB address
locations_MMS = locations_MMS[['Name_MMS','Address_MMS','City','State','Zip_MMS']].dropna(how='any')
locations_MMS['MMS>DB'] = False

# for every row in Address_MMS, check if there is a substring in Address_DB

for i in range(locations_MMS.shape[0]):
    address = str(locations_MMS.iloc[i,1])
    #print(address)
    for item in locations['Address_DB']:
        #print(item_DB)
        if str(item) in address:
            locations_MMS.loc[i,'Address_DB'] = item
            locations_MMS.loc[i,'MMS>DB'] = True
            #print('Found it!')
            break  

# take a look at locations that are in MMS but not in DB
print(locations_MMS['MMS>DB'].value_counts(dropna=False))
locations_MMS[locations_MMS['MMS>DB']==True].tail()

In [ ]:
# populate a new boolean column 'DB>MMS' that says 'True' if DB address contains MMS address
locations = locations.dropna(how='any')
locations['DB>MMS'] = False

#for address_DB in locations['Address_DB'], check if there is a substring in Address_MMS
for i in range (locations.shape[0]):
    item = str(locations.iloc[i,1])
    for address in locations_MMS['Address_MMS']:
        if str(address) in item:
            locations.loc[i,'Address_MMS'] = address
            locations.loc[i,'DB>MMS'] = True
            break

# take a look at locations that are in MMS but not in DB
#print(locations[locations['DB>MMS']==False])
print(locations['DB>MMS'].value_counts(dropna=False))
locations.head(10)
locations[locations['DB>MMS']==True]


In [ ]:
# outer merge with the street address
new_locations = locations_MMS.merge(locations, indicator=True, how='outer',
                                    left_on='Address_MMS',right_on='Address_DB')
# new_locations = new_locations[new_locations['_merge']=='both']
new_locations = new_locations[new_locations['_merge']=='left_only']
#new_locations = new_locations[(new_locations['_merge']=='left_only') | (new_locations['_merge']=='right_only')]
new_locations = new_locations.drop_duplicates(subset='Name_MMS')
new_locations.info()
new_locations.head(10)

#diff = df.merge(entities,indicator = True, how='left',on='Tax ID')
#        diff = diff[diff['_merge']=='left_only']
# 2. Put in a new dataframe `new_locations` (*Columns: Name_MMS, Name_DB, Address, City, State, Zip (for now)*)
# 3. Export the dataframe into a new .csv file `new_locations.csv`

## Note:

There is a way to compute the 'distance' between the two strings (i.e. [Levinshtein distance](https://www.datacamp.com/community/tutorials/fuzzy-string-python)) and choose a certain row if the distance between the two strings is short enough.
There's also the `fuzzywuzzy` package that computes the 'fuzz ratio' - how similar the two strings are. Great thing about this one is that it supports partial ratio (like a search) and mixed orders (i.e. 'US vs Canada' and 'Canada vs US' will have a token ratio of 100%).

But since we are looking for a definite solution (i.e. would rather extract more information than to miss some) I didn't use these packages.

### Just...exploring. random notes.

#### MMS First Impression
* The sheet `MACK` is in a format that is different from all other sheets for individual hospitals. Why...
* For all other sheets, there is a mini-table at the bottom with sum summary numbers.
* Minor variations in `Practice Address` (e.g. 'St' vs 'St.' vs 'Street'; 'Trail vs 'Tr.'; 'Carmel St.' vs 'S. Carmel St.')
* `Specialty` seems to be clean and standardized
* Ooh there is a column `Primary Facility` that has the hospital code as its value; I can download everything (except `MACK`) into one dataframe. 
    * not all of these will be useful
* I'm just going to believe the provider names, NPI, Tax ID, and phone numbers.

__*Practice column is both a location and an entity.__ 

#### Providers list First Impression
* `Provider Name: Last`, `Provider Name: First`, and `Provider NPI` will be useful.
* `Provider Primary Specialty` has some empty cells but still usable; just need to use in conjunction with other things
* There are subtle variations between `Primary Employer`(healthcare entity - TIN) and `Primary Practice Location` (healthcare location - Group NPI); also some are empty cells.
    * these are going away in the next couple of weeks, though.  

#### Locations list First Impression
* pretty messy...
* `Location Name` is almost clean; some entries have NPI attached at the end but we can remove it.
* Addresses are all messed up. Most have the physical address in full (`Physical Address`), but not all. Some entries don't have physical address but only state and zip code; Some have state code under the column `Physical Address: City`; the only column without any missing values seems to be `Physical Address: Zip`. 
    * __Adam's note: throw these out :)__ 
* There are information about latitudes, longitudes, and phone numbers; but they aren't available for all locations.

#### Entities list First Impression
* `Entity Legal Name` looks nice and clean. No empty cells; no weird variations (at least on the first look). This could be used as a standard.
* There are many entities that literally have no information other than their name and TIN...what to do? 
    * probably 
* There's the `Billing Address` and there's the `Mailing Address`, and they are _different_. Geez
* The only columns that are fully populated are `Entity`,`Entity Legal Name`, and `Entity TIN`. If I am going to use any other columns I would have to be careful.